In [1]:
import time
import logging
from kiteconnect import KiteTicker
logger = logging.getLogger()
logging.basicConfig(level=logging.DEBUG)
import datetime
from utils.RenkoUtil import *
from classes.DayEntry import Day
from classes.MinuteEntry import Minute
from classes.TradeEntry import Trade
from collections import deque
import thread
import csv

In [2]:
#initialize
money = 1
day = Day(0, money, False)
stepCount = 3
brickHeight = 0.1
renkoDeque = deque(maxlen=stepCount)
printRenkoDeque(renkoDeque)

DEBUG:root:empty queue


In [3]:
# def tickScript(tick):
#     verbose = False
#     global day
#     global renkoDeque
#     global stepCount
#     global brickHeight
#     logging.info('Inside tickfunction that one')
#     minute = Minute(tick['timestamp'], tick['last_price'])
#     renkoDeque, newRenko = generateRenko(minute, renkoDeque, brickHeight, stepCount, verbose)
#     printRenkoDeque(renkoDeque)
#     while newRenko:
#         if len(renkoDeque) == stepCount:
#     #             if marketOpen <= minute.time < marketClose:
#             logging.info("Boughtflag: {}".format(day.boughtFlag))
#             if day.boughtFlag == 0:
#                 #market enter logic
#                 if getEmotion(renkoDeque) == stepCount:
#                     #buy
#                     day.boughtFlag = 1
#                     day.currentTrade = Trade(minute.listPrice, minute.time, 0, 0, day.boughtFlag)
#                     logging.info('Long trade {}'.format(day.currentTrade.enterPrice))

#                 elif getEmotion(renkoDeque) == -1 * stepCount:
#                     #sell
#                     day.boughtFlag = -1
#                     day.currentTrade  = Trade(minute.listPrice, minute.time, 0, 0, day.boughtFlag)
#                     logging.info('Short trade {}'.format(day.currentTrade.enterPrice))

#             elif day.boughtFlag == 1:
#                 #market exit logic (when already bought)
#                 if renkoExitLogic(renkoDeque, stepCount, day.boughtFlag):
#                     logging.info('exiting trade, enter: {} exit: {}'.format(day.currentTrade.enterPrice, minute.listPrice))
#                     day.currentTrade.exitTrade('renkoExit', minute.time, minute.listPrice, verbose = True)
#                     day.addTrade(day.currentTrade)
#                     logging.info('Long exit')
#             else:
#                 #market exit logic (when already sold)
#                 if renkoExitLogic(renkoDeque, stepCount, day.boughtFlag):
#                     logging.info('exiting trade, enter: {} exit: {}'.format(day.currentTrade.enterPrice, minute.listPrice))
#                     day.currentTrade.exitTrade('renkoExit', minute.time, minute.listPrice, verbose = True)
#                     day.addTrade(day.currentTrade)
#                     logging.info('Short exit')
#         renkoDeque, newRenko = generateRenko(minute, renkoDeque, brickHeight, stepCount, verbose)
#     #         elif minute.time >= marketClose and day.sellEndOfDay:
#     #             if day.boughtFlag == 1:
#     #change to currentTrade
#     #                 trade.exitTrade('eod', minute.time, minute.closingPrice, verbose)
#     #                 day.addTrade(trade)
#     #             elif day.boughtFlag == -1:
#     #                 trade.exitTrade('eod', minute.time, minute.closingPrice, verbose)
#     #                 day.addTrade(trade)

In [4]:
def writeToCsv(ticks):
    newPoint = {'timestamp':ticks[0]['timestamp'], 
                'lp_1':ticks[0]['last_price']
                }
#     print(newPoint)
    with open('tick.csv', 'a') as f_out:
        out_colnames = ["timestamp","lp_1"]
        csv_writer = csv.DictWriter(f_out, fieldnames = out_colnames)
        csv_writer.writerow(newPoint)
    

In [5]:
with open('tick.csv', 'w') as f_out:
    out_colnames = ["timestamp","lp_1"]
    csv_writer = csv.DictWriter(f_out, fieldnames = out_colnames)
    csv_writer.writeheader()

kws = KiteTicker("5auxbio6wezhjhhb", "6fdabmjp7XzWcYZ5gPoKWbd4L0cJ1187")

def on_ticks(ws, ticks):
    # Callback to receive ticks.
#     print(time.time())
    logging.debug("Ticks: {} {}".format(ticks[0]['last_price'], ticks[0]['timestamp']))
    try:
        thread.start_new_thread(writeToCsv, (ticks,))
    except:
        logging.debug('Unable to start that one thread')
def on_connect(ws, response):
    # Callback on successful connect.
    # Subscribe to a list of instrument_tokens (RELIANCE and ACC here).
    ws.subscribe([6401])
#     ws.subscribe([6401, 857857, 738561])

    # Set RELIANCE to tick in `full` mode.
#     ws.set_mode(ws.MODE_FULL, [6401, 857857, 738561])
    ws.set_mode(ws.MODE_FULL, [6401])

def on_close(ws, code, reason):
    # On connection close stop the main loop
    # Reconnection will not happen after executing `ws.stop()`
    #ws.stop()
    pass

# Assign the callbacks.
kws.on_ticks = on_ticks
kws.on_connect = on_connect
kws.on_close = on_close

# Infinite loop on the main thread. Nothing after this will run.
# You have to use the pre-defined callbacks to manage subscriptions.
kws.connect(threaded=True)

DEBUG:root:Ticks: 116.7 2019-05-16 09:52:39
DEBUG:root:Ticks: 116.7 2019-05-16 09:52:42
DEBUG:root:Ticks: 116.7 2019-05-16 09:52:45
DEBUG:root:Ticks: 117.0 2019-05-16 09:52:45
DEBUG:root:Ticks: 117.0 2019-05-16 09:52:46
DEBUG:root:Ticks: 117.0 2019-05-16 09:52:47
DEBUG:root:Ticks: 116.8 2019-05-16 09:52:48
DEBUG:root:Ticks: 116.8 2019-05-16 09:52:49
DEBUG:root:Ticks: 116.8 2019-05-16 09:52:52
DEBUG:root:Ticks: 116.8 2019-05-16 09:52:54
DEBUG:root:Ticks: 116.8 2019-05-16 09:52:55
DEBUG:root:Ticks: 116.8 2019-05-16 09:52:55
DEBUG:root:Ticks: 116.75 2019-05-16 09:52:57
DEBUG:root:Ticks: 116.75 2019-05-16 09:52:58
DEBUG:root:Ticks: 116.75 2019-05-16 09:52:59
DEBUG:root:Ticks: 116.75 2019-05-16 09:53:00
DEBUG:root:Ticks: 116.75 2019-05-16 09:53:01
DEBUG:root:Ticks: 116.75 2019-05-16 09:53:02
DEBUG:root:Ticks: 116.75 2019-05-16 09:53:04
DEBUG:root:Ticks: 116.75 2019-05-16 09:53:05
DEBUG:root:Ticks: 116.75 2019-05-16 09:53:05
DEBUG:root:Ticks: 116.75 2019-05-16 09:53:06
DEBUG:root:Ticks: 116.

DEBUG:root:Ticks: 116.6 2019-05-16 09:56:30
DEBUG:root:Ticks: 116.5 2019-05-16 09:56:31
DEBUG:root:Ticks: 116.5 2019-05-16 09:56:33
DEBUG:root:Ticks: 116.6 2019-05-16 09:56:33
DEBUG:root:Ticks: 116.6 2019-05-16 09:56:34
DEBUG:root:Ticks: 116.6 2019-05-16 09:56:37
DEBUG:root:Ticks: 116.6 2019-05-16 09:56:37
DEBUG:root:Ticks: 116.6 2019-05-16 09:56:38
DEBUG:root:Ticks: 116.6 2019-05-16 09:56:39
DEBUG:root:Ticks: 116.6 2019-05-16 09:56:42
DEBUG:root:Ticks: 116.6 2019-05-16 09:56:46
DEBUG:root:Ticks: 116.6 2019-05-16 09:56:51
DEBUG:root:Ticks: 116.5 2019-05-16 09:56:52
DEBUG:root:Ticks: 116.5 2019-05-16 09:56:52
DEBUG:root:Ticks: 116.5 2019-05-16 09:56:53
DEBUG:root:Ticks: 116.6 2019-05-16 09:56:54
DEBUG:root:Ticks: 116.6 2019-05-16 09:56:55
DEBUG:root:Ticks: 116.6 2019-05-16 09:56:56
DEBUG:root:Ticks: 116.6 2019-05-16 09:56:57
DEBUG:root:Ticks: 116.6 2019-05-16 09:56:58
DEBUG:root:Ticks: 116.6 2019-05-16 09:56:59
DEBUG:root:Ticks: 116.6 2019-05-16 09:57:01
DEBUG:root:Ticks: 116.6 2019-05-

DEBUG:root:Ticks: 116.4 2019-05-16 10:01:17
DEBUG:root:Ticks: 116.4 2019-05-16 10:01:18
DEBUG:root:Ticks: 116.4 2019-05-16 10:01:19
DEBUG:root:Ticks: 116.4 2019-05-16 10:01:20
DEBUG:root:Ticks: 116.4 2019-05-16 10:01:21
DEBUG:root:Ticks: 116.4 2019-05-16 10:01:22
DEBUG:root:Ticks: 116.45 2019-05-16 10:01:23
DEBUG:root:Ticks: 116.45 2019-05-16 10:01:23
DEBUG:root:Ticks: 116.4 2019-05-16 10:01:24
DEBUG:root:Ticks: 116.4 2019-05-16 10:01:25
DEBUG:root:Ticks: 116.4 2019-05-16 10:01:27
DEBUG:root:Ticks: 116.4 2019-05-16 10:01:29
DEBUG:root:Ticks: 116.4 2019-05-16 10:01:31
DEBUG:root:Ticks: 116.35 2019-05-16 10:01:31
DEBUG:root:Ticks: 116.35 2019-05-16 10:01:32
DEBUG:root:Ticks: 116.35 2019-05-16 10:01:34
DEBUG:root:Ticks: 116.35 2019-05-16 10:01:36
DEBUG:root:Ticks: 116.35 2019-05-16 10:01:38
DEBUG:root:Ticks: 116.35 2019-05-16 10:01:39
DEBUG:root:Ticks: 116.35 2019-05-16 10:01:41
DEBUG:root:Ticks: 116.35 2019-05-16 10:01:42
DEBUG:root:Ticks: 116.4 2019-05-16 10:01:44
DEBUG:root:Ticks: 116.

DEBUG:root:Ticks: 116.45 2019-05-16 10:06:19
DEBUG:root:Ticks: 116.45 2019-05-16 10:06:20
DEBUG:root:Ticks: 116.5 2019-05-16 10:06:21
DEBUG:root:Ticks: 116.45 2019-05-16 10:06:21
DEBUG:root:Ticks: 116.45 2019-05-16 10:06:22
DEBUG:root:Ticks: 116.45 2019-05-16 10:06:25
DEBUG:root:Ticks: 116.45 2019-05-16 10:06:27
DEBUG:root:Ticks: 116.4 2019-05-16 10:06:28
DEBUG:root:Ticks: 116.4 2019-05-16 10:06:29
DEBUG:root:Ticks: 116.4 2019-05-16 10:06:30
DEBUG:root:Ticks: 116.4 2019-05-16 10:06:31
DEBUG:root:Ticks: 116.4 2019-05-16 10:06:32
DEBUG:root:Ticks: 116.4 2019-05-16 10:06:34
DEBUG:root:Ticks: 116.4 2019-05-16 10:06:36
DEBUG:root:Ticks: 116.4 2019-05-16 10:06:37
DEBUG:root:Ticks: 116.45 2019-05-16 10:06:38
DEBUG:root:Ticks: 116.45 2019-05-16 10:06:40
DEBUG:root:Ticks: 116.45 2019-05-16 10:06:44
DEBUG:root:Ticks: 116.45 2019-05-16 10:06:45
DEBUG:root:Ticks: 116.45 2019-05-16 10:06:45
DEBUG:root:Ticks: 116.45 2019-05-16 10:06:47
DEBUG:root:Ticks: 116.35 2019-05-16 10:06:48
DEBUG:root:Ticks: 1

DEBUG:root:Ticks: 116.25 2019-05-16 10:10:24
DEBUG:root:Ticks: 116.3 2019-05-16 10:10:25
DEBUG:root:Ticks: 116.3 2019-05-16 10:10:28
DEBUG:root:Ticks: 116.3 2019-05-16 10:10:29
DEBUG:root:Ticks: 116.25 2019-05-16 10:10:29
DEBUG:root:Ticks: 116.25 2019-05-16 10:10:29
DEBUG:root:Ticks: 116.25 2019-05-16 10:10:33
DEBUG:root:Ticks: 116.25 2019-05-16 10:10:34
DEBUG:root:Ticks: 116.25 2019-05-16 10:10:37
DEBUG:root:Ticks: 116.25 2019-05-16 10:10:37
DEBUG:root:Ticks: 116.25 2019-05-16 10:10:38
DEBUG:root:Ticks: 116.3 2019-05-16 10:10:40
DEBUG:root:Ticks: 116.3 2019-05-16 10:10:42
DEBUG:root:Ticks: 116.3 2019-05-16 10:10:43
DEBUG:root:Ticks: 116.3 2019-05-16 10:10:45
DEBUG:root:Ticks: 116.3 2019-05-16 10:10:45
DEBUG:root:Ticks: 116.2 2019-05-16 10:10:48
DEBUG:root:Ticks: 116.2 2019-05-16 10:10:49
DEBUG:root:Ticks: 116.2 2019-05-16 10:10:50
DEBUG:root:Ticks: 116.25 2019-05-16 10:10:50
DEBUG:root:Ticks: 116.25 2019-05-16 10:10:51
DEBUG:root:Ticks: 116.25 2019-05-16 10:10:52
DEBUG:root:Ticks: 116

...

DEBUG:root:Ticks: 119.0 2019-05-16 15:15:29
DEBUG:root:Ticks: 119.0 2019-05-16 15:15:29
DEBUG:root:Ticks: 118.85 2019-05-16 15:15:31
DEBUG:root:Ticks: 118.85 2019-05-16 15:15:32
DEBUG:root:Ticks: 118.9 2019-05-16 15:15:33
DEBUG:root:Ticks: 118.8 2019-05-16 15:15:34
DEBUG:root:Ticks: 118.8 2019-05-16 15:15:35
DEBUG:root:Ticks: 118.9 2019-05-16 15:15:37
DEBUG:root:Ticks: 118.9 2019-05-16 15:15:38
DEBUG:root:Ticks: 118.9 2019-05-16 15:15:39
DEBUG:root:Ticks: 118.9 2019-05-16 15:15:40
DEBUG:root:Ticks: 118.9 2019-05-16 15:15:41
DEBUG:root:Ticks: 118.8 2019-05-16 15:15:42
DEBUG:root:Ticks: 118.9 2019-05-16 15:15:43
DEBUG:root:Ticks: 118.9 2019-05-16 15:15:44
DEBUG:root:Ticks: 118.9 2019-05-16 15:15:46
DEBUG:root:Ticks: 118.9 2019-05-16 15:15:48
DEBUG:root:Ticks: 118.9 2019-05-16 15:15:49
DEBUG:root:Ticks: 118.8 2019-05-16 15:15:50
DEBUG:root:Ticks: 118.8 2019-05-16 15:15:51
DEBUG:root:Ticks: 118.8 2019-05-16 15:15:52
DEBUG:root:Ticks: 118.8 2019-05-16 15:15:54
DEBUG:root:Ticks: 118.9 2019-0

DEBUG:root:Ticks: 118.7 2019-05-16 15:18:33
DEBUG:root:Ticks: 118.7 2019-05-16 15:18:35
DEBUG:root:Ticks: 118.7 2019-05-16 15:18:36
DEBUG:root:Ticks: 118.65 2019-05-16 15:18:37
DEBUG:root:Ticks: 118.6 2019-05-16 15:18:38
DEBUG:root:Ticks: 118.65 2019-05-16 15:18:39
DEBUG:root:Ticks: 118.65 2019-05-16 15:18:39
DEBUG:root:Ticks: 118.6 2019-05-16 15:18:40
DEBUG:root:Ticks: 118.6 2019-05-16 15:18:41
DEBUG:root:Ticks: 118.85 2019-05-16 15:18:42
DEBUG:root:Ticks: 118.85 2019-05-16 15:18:42
DEBUG:root:Ticks: 118.85 2019-05-16 15:18:43
DEBUG:root:Ticks: 118.85 2019-05-16 15:18:44
DEBUG:root:Ticks: 118.8 2019-05-16 15:18:45
DEBUG:root:Ticks: 118.8 2019-05-16 15:18:45
DEBUG:root:Ticks: 118.85 2019-05-16 15:18:46
DEBUG:root:Ticks: 118.7 2019-05-16 15:18:48
DEBUG:root:Ticks: 118.7 2019-05-16 15:18:48
DEBUG:root:Ticks: 118.7 2019-05-16 15:18:50
DEBUG:root:Ticks: 118.7 2019-05-16 15:18:51
DEBUG:root:Ticks: 118.7 2019-05-16 15:18:51
DEBUG:root:Ticks: 118.7 2019-05-16 15:18:52
DEBUG:root:Ticks: 118.85

DEBUG:root:Ticks: 118.95 2019-05-16 15:21:45
DEBUG:root:Ticks: 119.0 2019-05-16 15:21:46
DEBUG:root:Ticks: 119.0 2019-05-16 15:21:47
DEBUG:root:Ticks: 118.95 2019-05-16 15:21:48
DEBUG:root:Ticks: 119.0 2019-05-16 15:21:49
DEBUG:root:Ticks: 119.0 2019-05-16 15:21:50
DEBUG:root:Ticks: 119.25 2019-05-16 15:21:51
DEBUG:root:Ticks: 119.25 2019-05-16 15:21:52
DEBUG:root:Ticks: 119.25 2019-05-16 15:21:53
DEBUG:root:Ticks: 119.3 2019-05-16 15:21:55
DEBUG:root:Ticks: 119.25 2019-05-16 15:21:55
DEBUG:root:Ticks: 119.25 2019-05-16 15:21:56
DEBUG:root:Ticks: 119.25 2019-05-16 15:21:57
DEBUG:root:Ticks: 119.3 2019-05-16 15:21:58
DEBUG:root:Ticks: 119.3 2019-05-16 15:21:59
DEBUG:root:Ticks: 119.25 2019-05-16 15:22:00
DEBUG:root:Ticks: 119.25 2019-05-16 15:22:00
DEBUG:root:Ticks: 119.3 2019-05-16 15:22:01
DEBUG:root:Ticks: 119.3 2019-05-16 15:22:02
DEBUG:root:Ticks: 119.25 2019-05-16 15:22:02
DEBUG:root:Ticks: 119.25 2019-05-16 15:22:04
DEBUG:root:Ticks: 119.3 2019-05-16 15:22:04
DEBUG:root:Ticks: 11

DEBUG:root:Ticks: 118.9 2019-05-16 15:25:18
DEBUG:root:Ticks: 119.0 2019-05-16 15:25:19
DEBUG:root:Ticks: 119.0 2019-05-16 15:25:19
DEBUG:root:Ticks: 119.0 2019-05-16 15:25:22
DEBUG:root:Ticks: 118.85 2019-05-16 15:25:23
DEBUG:root:Ticks: 119.0 2019-05-16 15:25:24
DEBUG:root:Ticks: 119.0 2019-05-16 15:25:24
DEBUG:root:Ticks: 119.0 2019-05-16 15:25:25
DEBUG:root:Ticks: 119.0 2019-05-16 15:25:26
DEBUG:root:Ticks: 119.0 2019-05-16 15:25:27
DEBUG:root:Ticks: 119.0 2019-05-16 15:25:28
DEBUG:root:Ticks: 118.85 2019-05-16 15:25:29
DEBUG:root:Ticks: 118.95 2019-05-16 15:25:30
DEBUG:root:Ticks: 118.95 2019-05-16 15:25:30
DEBUG:root:Ticks: 118.95 2019-05-16 15:25:32
DEBUG:root:Ticks: 118.95 2019-05-16 15:25:33
DEBUG:root:Ticks: 118.95 2019-05-16 15:25:34
DEBUG:root:Ticks: 118.95 2019-05-16 15:25:37
DEBUG:root:Ticks: 118.9 2019-05-16 15:25:39
DEBUG:root:Ticks: 118.9 2019-05-16 15:25:40
DEBUG:root:Ticks: 118.85 2019-05-16 15:25:41
DEBUG:root:Ticks: 118.85 2019-05-16 15:25:43
DEBUG:root:Ticks: 118.

DEBUG:root:Ticks: 118.9 2019-05-16 15:28:37
DEBUG:root:Ticks: 118.9 2019-05-16 15:28:38
DEBUG:root:Ticks: 118.9 2019-05-16 15:28:39
DEBUG:root:Ticks: 118.9 2019-05-16 15:28:41
DEBUG:root:Ticks: 118.9 2019-05-16 15:28:42
DEBUG:root:Ticks: 118.9 2019-05-16 15:28:43
DEBUG:root:Ticks: 118.9 2019-05-16 15:28:45
DEBUG:root:Ticks: 118.9 2019-05-16 15:28:46
DEBUG:root:Ticks: 118.7 2019-05-16 15:28:47
DEBUG:root:Ticks: 118.9 2019-05-16 15:28:48
DEBUG:root:Ticks: 118.9 2019-05-16 15:28:49
DEBUG:root:Ticks: 118.9 2019-05-16 15:28:50
DEBUG:root:Ticks: 118.9 2019-05-16 15:28:52
DEBUG:root:Ticks: 118.9 2019-05-16 15:28:53
DEBUG:root:Ticks: 118.9 2019-05-16 15:28:55
DEBUG:root:Ticks: 118.7 2019-05-16 15:28:55
DEBUG:root:Ticks: 118.7 2019-05-16 15:28:56
DEBUG:root:Ticks: 118.7 2019-05-16 15:28:58
DEBUG:root:Ticks: 118.7 2019-05-16 15:28:59
DEBUG:root:Ticks: 118.65 2019-05-16 15:29:00
DEBUG:root:Ticks: 118.9 2019-05-16 15:29:01
DEBUG:root:Ticks: 118.9 2019-05-16 15:29:03
DEBUG:root:Ticks: 118.9 2019-05